In [1]:
from LiveAMP import *
# run_amp(cycle_day=98)
self = AMP(cycle_day=98, param=param_lst[0])
# #     # crse_code='ansc1319',
# #     # crse_code='an/th2302',
    # )
# self.get_y_pred()
# self.get_X_proc()
# self.get_terms()
self.get_X()
# Z = self.X.reset_index().merge(self.y_pred.reset_index())


AMP(overwrite={'stack'}, dependence={'adm': 'raw', 'flg': 'raw', 'raw': 'X', 'reg': 'X', 'X': 'X_proc', 'X_proc': 'Y'}, cycle_day=98, proj_code=202408, term_codes=(202108, 202208, 202308, 202408), crse_code='_anycrse', styp_code='n', stats=('mean',), show=set(), param={'trf': ['2d76', "[('_allcrse','passthrough',['_allcrse_']),('act_equiv','passthrough',['act_equiv_']),('act_equiv_missing','passthrough',['act_equiv_missing_']),('apdc_day','passthrough',['apdc_day_']),('birth_day','passthrough',['birth_day_']),('coll_code','passthrough',['coll_code_']),('distance','passthrough',['distance_']),('gap_score','passthrough',['gap_score_']),('gender','passthrough',['gender_']),('hs_qrtl','passthrough',['hs_qrtl_']),('international','passthrough',['international_']),('lgcy','passthrough',['lgcy_']),('math','passthrough',['math_']),('oriented','passthrough',['oriented_']),('race_american_indian','passthrough',['race_american_indian_']),('race_asian','passthrough',['race_asian_']),('race_black',

In [2]:
self.root_path

PosixPath('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/098')

In [42]:
# path = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/098/y_pred')
grp = [variable,'pidm','id','levl_code','styp_code','term_code','train_code','sim','trf_hash','imp_hash','clf_hash']
# y_pred = pd.concat([read(fn).get('y_pred', pd.DataFrame()) for fn in sorted(path.rglob('*.pkl'))])
y_pred = pd.concat([read(fn).get('y_pred', pd.DataFrame()) for fn in sorted((self.root_path / 'y_pred').rglob('*.pkl'))])
Z = self.X.reset_index().merge(y_pred.reset_index()).prep().set_index(grp)
grp.remove('pidm')
grp.remove('id')

In [47]:
Z.query("crse_code=='_anycrse'")

admit  \
coll_code pidm    id      levl_code styp_code term_code train_code sim trf_hash imp_hash clf_hash          
an        9726662 9726457 ug        n         202108    202108     0   2d76     77d7     fb12       True   
                                                                   1   2d76     77d7     fb12       True   
                                                                   2   2d76     77d7     fb12       True   
                                                                   3   2d76     77d7     fb12       True   
                                                                   4   2d76     77d7     fb12       True   
...                                                                                                  ...   
ba        9240905 9247845 ug        n         202408    202408     5   2d76     77d7     fb12       True   
                                                                   6   2d76     77d7     fb12       True   
                                                                   7   2d76     77d7     fb12       True   
                                                                   8   2d76     77d7     fb12       True   
                                                                   9   2d76     77d7     fb12       True   

                                                                                                   enroll  \
coll_code pidm    id      levl_code styp_code term_code train_code sim trf_hash imp_hash clf_hash           
an        9726662 9726457 ug        n         202108    202108     0   2d76     77d7     fb12       False   
                                                                   1   2d76     77d7     fb12       False   
                                                                   2   2d76     77d7     fb12       False   
                                                                   3   2d76     77d7     fb12       False   
                                                                   4   2d76     77d7     fb12       False   
...                                                                                                   ...   
ba        9240905 9247845 ug        n         202408    202408     5   2d76     77d7     fb12       False   
                                                                   6   2d76     77d7     fb12       False   
                                                                   7   2d76     77d7     fb12       False   
                                                                   8   2d76     77d7     fb12       False   
                                                                   9   2d76     77d7     fb12       False   

                                                                                                   matric  \
coll_code pidm    id      levl_code styp_code term_code train_code sim trf_hash imp_hash clf_hash           
an        9726662 9726457 ug        n         202108    202108     0   2d76     77d7     fb12       False   
                                                                   1   2d76     77d7     fb12       False   
                                                                   2   2d76     77d7     fb12       False   
                                                                   3   2d76     77d7     fb12       False   
                                                                   4   2d76     77d7     fb12       False   
...                                                                                                   ...   
ba        9240905 9247845 ug        n         202408    202408     5   2d76     77d7     fb12       False   
                                                                   6   2d76     77d7     fb12       False   
                                                                   7   2d76     77d7     fb12       False   
                                                                   8   2d76  

In [46]:
variable = 'crse_code'
# variable = 'coll_code'
S = (Z if variable == 'crse_code' else Z.query("crse_code=='_anycrse'")
    .groupby(grp).apply(lambda y: pd.Series({
            'predict': y['proba'].sum(),
            'test_score': log_loss(y['actual'], y['proba'], labels=[False,True]) * 100,
        }), include_groups=False)
    .join(self.train_score)
    .join(self.agg.loc[variable].rename_axis(index={'value':variable}))
)
P = S['actual'].rename('prior').reset_index()
P['term_code'] += 100
S = S.reset_index().merge(P, 'left')
alpha = 1
S['overall_score'] = (S['train_score'] + alpha * S['test_score']) / (1 + alpha)
S['predict'] *= S['mlt']
S['predict_error'] = S['predict'] - S['actual']
S['predict_error_pct'] = S['predict_error'] / S['actual'] * 100
S['predict_change'] = S['predict'] - S['prior']
S['predict_change_pct'] = S['predict_change'] / S['prior'] * 100
S['train_code'] = S['train_code'].astype('string').replace(str(self.proj_code), 'all')
S = (S
    .prep()
    .sort_values(grp, ascending=[True,True,True,False,False,True,True,True,True])
    .set_index(grp)
    [['admit','enroll','predict','prior','predict_change','predict_change_pct','actual','predict_error','predict_error_pct','overall_score','test_score','train_score','mlt']]
)
grp.remove('sim')
M = S.groupby(grp, sort=False).mean().prep()
S

KeyboardInterrupt: 

In [41]:
path = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/098/y_pred')
y_pred = pd.concat([read(fn).get('y_pred', pd.DataFrame()) for fn in sorted(path.rglob('*.pkl'))])
# dct = {str(fn): read(fn).get('y_pred', pd.DataFrame()) for fn in sorted(path.rglob('*.pkl'))}
# dct = {str(fn): read(fn)['y_pred'] for fn in sorted(path.rglob('*.pkl'))[:20]}
# dct = [read(fn) for fn in sorted(path.rglob('*.pkl'))[:2]]

In [40]:
pd.concat(dct.values())

actual  \
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash           
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12        True   
                                        1   2d76     77d7     fb12        True   
                                        2   2d76     77d7     fb12        True   
                                        3   2d76     77d7     fb12        True   
                                        4   2d76     77d7     fb12        True   
...                                                                        ...   
univ0342  202208    9175041  202408     5   2d76     77d7     fb12       False   
                                        6   2d76     77d7     fb12       False   
                                        7   2d76     77d7     fb12       False   
                                        8   2d76     77d7     fb12       False   
                                        9   2d76     77d7     fb12       False   

                                                                         pred  \
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash          
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12       True   
                                        1   2d76     77d7     fb12       True   
                                        2   2d76     77d7     fb12       True   
                                        3   2d76     77d7     fb12       True   
                                        4   2d76     77d7     fb12       True   
...                                                                       ...   
univ0342  202208    9175041  202408     5   2d76     77d7     fb12      False   
                                        6   2d76     77d7     fb12      False   
                                        7   2d76     77d7     fb12      False   
                                        8   2d76     77d7     fb12      False   
                                        9   2d76     77d7     fb12      False   

                                                                           proba  
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash            
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12      0.999593  
                                        1   2d76     77d7     fb12      0.999598  
                                        2   2d76     77d7     fb12      0.999451  
                                        3   2d76     77d7     fb12      0.999593  
                                        4   2d76     77d7     fb12      0.999451  
...                                                                          ...  
univ0342  202208    9175041  202408     5   2d76     77d7     fb12      0.000447  
                                        6   2d76     77d7     fb12      0.000447  
                                        7   2d76     77d7     fb12      0.000447  
                                        8   2d76     77d7     fb12      0.000447  
                                        9   2d76     77d7     fb12      0.000447  

[79142450 rows x 3 columns]

In [31]:
dct[0].get('y_pred', pd.NA)

actual  \
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash           
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12        True   
                                        1   2d76     77d7     fb12        True   
                                        2   2d76     77d7     fb12        True   
                                        3   2d76     77d7     fb12        True   
                                        4   2d76     77d7     fb12        True   
...                                                                        ...   
          202208    9175041  202408     5   2d76     77d7     fb12       False   
                                        6   2d76     77d7     fb12       False   
                                        7   2d76     77d7     fb12       False   
                                        8   2d76     77d7     fb12       False   
                                        9   2d76     77d7     fb12       False   

                                                                         pred  \
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash          
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12       True   
                                        1   2d76     77d7     fb12       True   
                                        2   2d76     77d7     fb12       True   
                                        3   2d76     77d7     fb12       True   
                                        4   2d76     77d7     fb12       True   
...                                                                       ...   
          202208    9175041  202408     5   2d76     77d7     fb12      False   
                                        6   2d76     77d7     fb12      False   
                                        7   2d76     77d7     fb12      False   
                                        8   2d76     77d7     fb12      False   
                                        9   2d76     77d7     fb12      False   

                                                                           proba  
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash            
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12      0.999593  
                                        1   2d76     77d7     fb12      0.999598  
                                        2   2d76     77d7     fb12      0.999451  
                                        3   2d76     77d7     fb12      0.999593  
                                        4   2d76     77d7     fb12      0.999451  
...                                                                          ...  
          202208    9175041  202408     5   2d76     77d7     fb12      0.012217  
                                        6   2d76     77d7     fb12      0.012217  
                                        7   2d76     77d7     fb12      0.012217  
                                        8   2d76     77d7     fb12      0.012217  
                                        9   2d76     77d7     fb12      0.012217  

[1350080 rows x 3 columns]

In [21]:
self.y_pred

actual  \
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash           
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12        True   
                                        1   2d76     77d7     fb12        True   
                                        2   2d76     77d7     fb12        True   
                                        3   2d76     77d7     fb12        True   
                                        4   2d76     77d7     fb12        True   
...                                                                        ...   
          202208    9175041  202408     5   2d76     77d7     fb12       False   
                                        6   2d76     77d7     fb12       False   
                                        7   2d76     77d7     fb12       False   
                                        8   2d76     77d7     fb12       False   
                                        9   2d76     77d7     fb12       False   

                                                                         pred  \
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash          
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12       True   
                                        1   2d76     77d7     fb12       True   
                                        2   2d76     77d7     fb12       True   
                                        3   2d76     77d7     fb12       True   
                                        4   2d76     77d7     fb12       True   
...                                                                       ...   
          202208    9175041  202408     5   2d76     77d7     fb12      False   
                                        6   2d76     77d7     fb12      False   
                                        7   2d76     77d7     fb12      False   
                                        8   2d76     77d7     fb12      False   
                                        9   2d76     77d7     fb12      False   

                                                                           proba  
crse_code term_code pidm     train_code sim trf_hash imp_hash clf_hash            
_anycrse  202308    10472212 202108     0   2d76     77d7     fb12      0.999593  
                                        1   2d76     77d7     fb12      0.999598  
                                        2   2d76     77d7     fb12      0.999451  
                                        3   2d76     77d7     fb12      0.999593  
                                        4   2d76     77d7     fb12      0.999451  
...                                                                          ...  
          202208    9175041  202408     5   2d76     77d7     fb12      0.012217  
                                        6   2d76     77d7     fb12      0.012217  
                                        7   2d76     77d7     fb12      0.012217  
                                        8   2d76     77d7     fb12      0.012217  
                                        9   2d76     77d7     fb12      0.012217  

[1350080 rows x 3 columns]

In [6]:
# grp.remove('sim')
S.groupby(grp).mean().prep()

admit  \
coll_code levl_code styp_code term_code train_code trf_hash imp_hash clf_hash          
an        ug        n         202108    202108     2d76     77d7     fb12       1346   
                                        202208     2d76     77d7     fb12       1346   
                                        202308     2d76     77d7     fb12       1346   
                                        all        2d76     77d7     fb12       1346   
                              202208    202108     2d76     77d7     fb12       1618   
...                                                                              ...   
sm        ug        n         202308    all        2d76     77d7     fb12        898   
                              202408    202108     2d76     77d7     fb12       1355   
                                        202208     2d76     77d7     fb12       1355   
                                        202308     2d76     77d7     fb12       1355   
                                        all        2d76     77d7     fb12       1355   

                                                                               enroll  \
coll_code levl_code styp_code term_code train_code trf_hash imp_hash clf_hash           
an        ug        n         202108    202108     2d76     77d7     fb12         412   
                                        202208     2d76     77d7     fb12         412   
                                        202308     2d76     77d7     fb12         412   
                                        all        2d76     77d7     fb12         412   
                              202208    202108     2d76     77d7     fb12         313   
...                                                                               ...   
sm        ug        n         202308    all        2d76     77d7     fb12         109   
                              202408    202108     2d76     77d7     fb12         111   
                                        202208     2d76     77d7     fb12         111   
                                        202308     2d76     77d7     fb12         111   
                                        all        2d76     77d7     fb12         111   

                                                                                  predict  \
coll_code levl_code styp_code term_code train_code trf_hash imp_hash clf_hash               
an        ug        n         202108    202108     2d76     77d7     fb12      550.828283   
                                        202208     2d76     77d7     fb12       564.45123   
                                        202308     2d76     77d7     fb12      539.115518   
                                        all        2d76     77d7     fb12      558.174226   
                              202208    202108     2d76     77d7     fb12      598.492124   
...                                                                                   ...   
sm        ug        n         202308    all        2d76     77d7     fb12       255.52663   
                              202408    202108     2d76     77d7     fb12      229.285522   
                                        202208     2d76     77d7     fb12      248.277642   
                                        202308     2d76     77d7     fb12      246.710865   
                                        all        2d76     77d7     fb12      243.572645   

                                                                               prior  \
coll_code levl_code styp_code term_code train_code trf_hash imp_hash clf_hash          
an        ug        n         202108    202108     2d76     77d7     fb12       <NA>   
                                        202208     2d76     77d7     fb12       <NA>   
                                        202308     2d76     77d7     fb12       <NA>   
                                        all        2d76     77d7     fb12       <NA>   
                              202208  

In [3]:
variable = 'crse_code'
variable = 'coll_code'
grp = [variable,'term_code','levl_code','styp_code','train_code','sim','trf_hash','imp_hash','clf_hash']
S = (Z
    .groupby(grp).apply(lambda y: pd.Series({
            'predict': y['proba'].sum(),
            'test_score': log_loss(y['actual'], y['proba'], labels=[False,True]) * 100,
        }), include_groups=False)
    .join(self.train_score)
    .join(self.agg.loc[variable].rename_axis(index={'value':variable}))
    .rename(index={202408:'all'}, level='train_code')
)
alpha = 1
S['overall_score'] = (S['train_score'] + alpha * S['test_score']) / (1 + alpha)
S['predict'] *= S['mlt']
S['error'] = S['predict'] - S['actual']
S['error_pct'] = S['error'] / S['actual'] * 100
P = S.reset_index()
P['term_code'] += 100
S = S.join(P.set_index(S.index.names)['actual'].rename('prior'))
S['change'] = S['predict'] - S['prior']
S['change_pct'] = S['change'] / S['prior'] * 100
S = S[['admit','enroll','predict','actual','error','error_pct','prior','change','change_pct','overall_score','test_score','train_score','mlt']]

In [65]:
S

admit  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash           
an        202108    ug        n         202108     0   2d76     77d7     fb12      1346.0   
                                                   1   2d76     77d7     fb12      1346.0   
                                                   2   2d76     77d7     fb12      1346.0   
                                                   3   2d76     77d7     fb12      1346.0   
                                                   4   2d76     77d7     fb12      1346.0   
...                                                                                   ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12      1355.0   
                                                                         fb12      1355.0   
                                                   9   2d76     77d7     fb12      1355.0   
                                                                         fb12      1355.0   
                                                                         fb12      1355.0   

                                                                                   enroll  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash           
an        202108    ug        n         202108     0   2d76     77d7     fb12       412.0   
                                                   1   2d76     77d7     fb12       412.0   
                                                   2   2d76     77d7     fb12       412.0   
                                                   3   2d76     77d7     fb12       412.0   
                                                   4   2d76     77d7     fb12       412.0   
...                                                                                   ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12       111.0   
                                                                         fb12       111.0   
                                                   9   2d76     77d7     fb12       111.0   
                                                                         fb12       111.0   
                                                                         fb12       111.0   

                                                                                      predict  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash               
an        202108    ug        n         202108     0   2d76     77d7     fb12      550.685515   
                                                   1   2d76     77d7     fb12      550.919408   
                                                   2   2d76     77d7     fb12      550.932971   
                                                   3   2d76     77d7     fb12      550.798991   
                                                   4   2d76     77d7     fb12      550.819814   
...                                                                                       ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12      241.844098   
                                                                         fb12      241.035136   
                                                   9   2d76     77d7     fb12      247.872296   
                                                                         fb12      241.946689   
                                                                         fb12      241.137383   

                                                                                   actual  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash           
an        202108    ug        n         202108     0   2d76     77d7     fb12       554.0   
                                                   1   2d76     77d7     fb12       554.0   
                              

In [62]:
# S.index.set_level_values(S.index.get_level_values('train_code'), level='train_code')
# S.index.set_levels(S.index.get_level_values('term_code').tolist(), level='term_code')
# S.index.get_level_values('term_code').tolist()
# S.index['term_code']
# S.ge


predict  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash               
an        202108    ug        n         202108     0   2d76     77d7     fb12      550.685515   
                                                   1   2d76     77d7     fb12      550.919408   
                                                   2   2d76     77d7     fb12      550.932971   
                                                   3   2d76     77d7     fb12      550.798991   
                                                   4   2d76     77d7     fb12      550.819814   
...                                                                                       ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12      241.844098   
                                                                         fb12      241.035136   
                                                   9   2d76     77d7     fb12      247.872296   
                                                                         fb12      241.946689   
                                                                         fb12      241.137383   

                                                                                   test_score  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash               
an        202108    ug        n         202108     0   2d76     77d7     fb12       13.958115   
                                                   1   2d76     77d7     fb12       14.002534   
                                                   2   2d76     77d7     fb12       13.970920   
                                                   3   2d76     77d7     fb12       13.938600   
                                                   4   2d76     77d7     fb12       13.996183   
...                                                                                       ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12       76.653052   
                                                                         fb12       76.653052   
                                                   9   2d76     77d7     fb12       76.654091   
                                                                         fb12       76.654091   
                                                                         fb12       76.654091   

                                                                                   train_score  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash                
an        202108    ug        n         202108     0   2d76     77d7     fb12        13.574245   
                                                   1   2d76     77d7     fb12        13.574245   
                                                   2   2d76     77d7     fb12        13.574245   
                                                   3   2d76     77d7     fb12        13.574245   
                                                   4   2d76     77d7     fb12        13.574245   
...                                                                                        ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12        14.860527   
                                                                         fb12        14.860527   
                                                   9   2d76     77d7     fb12        14.860527   
                                                                         fb12        14.860527   
                                                                         fb12        14.860527   

                                                                                    admit  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash           
an        202108    ug        n         202108     0   2d76     77d7     fb12      1346.0   
        

In [24]:
self.X.rename(columns={'fall 2024':'a'}, level='term_desc')
self.X.replace

KeyError: 'Requested level (term_desc) does not match index name (variable)'

In [19]:
S.rename(index={202408:"all"}, level='train_code')

predict  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash               
an        202108    ug        n         202108     0   2d76     77d7     fb12      550.685515   
                                                   1   2d76     77d7     fb12      550.919408   
                                                   2   2d76     77d7     fb12      550.932971   
                                                   3   2d76     77d7     fb12      550.798991   
                                                   4   2d76     77d7     fb12      550.819814   
...                                                                                       ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12      241.844098   
                                                                         fb12      241.035136   
                                                   9   2d76     77d7     fb12      247.872296   
                                                                         fb12      241.946689   
                                                                         fb12      241.137383   

                                                                                   test_score  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash               
an        202108    ug        n         202108     0   2d76     77d7     fb12       13.958115   
                                                   1   2d76     77d7     fb12       14.002534   
                                                   2   2d76     77d7     fb12       13.970920   
                                                   3   2d76     77d7     fb12       13.938600   
                                                   4   2d76     77d7     fb12       13.996183   
...                                                                                       ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12       76.653052   
                                                                         fb12       76.653052   
                                                   9   2d76     77d7     fb12       76.654091   
                                                                         fb12       76.654091   
                                                                         fb12       76.654091   

                                                                                   train_score  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash                
an        202108    ug        n         202108     0   2d76     77d7     fb12        13.574245   
                                                   1   2d76     77d7     fb12        13.574245   
                                                   2   2d76     77d7     fb12        13.574245   
                                                   3   2d76     77d7     fb12        13.574245   
                                                   4   2d76     77d7     fb12        13.574245   
...                                                                                        ...   
sm        202408    ug        n         all        8   2d76     77d7     fb12        14.860527   
                                                                         fb12        14.860527   
                                                   9   2d76     77d7     fb12        14.860527   
                                                                         fb12        14.860527   
                                                                         fb12        14.860527   

                                                                                    admit  \
coll_code term_code levl_code styp_code train_code sim trf_hash imp_hash clf_hash           
an        202108    ug        n         202108     0   2d76     77d7     fb12      1346.0   
        

In [ ]:
A = self.agg.query(f"variable==@self.crse_code")
mask = A.eval(f"term_code!={self.proj_code}")
B = A[mask]
C = A[~mask].drop(columns='mlt').join(B['mlt'].rename(index={k:self.proj_code for k in self.term_codes}))
M = pd.concat([B,C])
# C
# A = self.agg.query('term_code==202408').drop(columns='mlt')
# B = self.agg.query('term_code!=202408')['mlt'].rename(index={k:self.proj_code for k in self.term_codes})
# C = A.join(B)
# C.reset_index().missing()
# C
# .rename(index={k:self.proj_code for k in self.term_codes})
# .rename(index={k*100+8:202408 for k in range(2021,2025)})
# .rename(index={202108:202408, 202208:202408, 202308:202408 for k in range(2021,2025)})
# B.vc('term_code')
# A.shape, B.shape, A.join(B).shape
# A.shape, A.join(B).shape
# A.join(B)

In [ ]:
Z

In [ ]:
variable = 'crse_code'
grp = [variable,'term_code','levl_code','styp_code','train_code','sim','trf_hash','imp_hash','clf_hash']
S = (Z
    .groupby(grp).apply(lambda y: pd.Series({
            'predict': y['proba'].sum(),
            'test_score': log_loss(y['actual'], y['proba'], labels=[False,True]) * 100,
        }), include_groups=False)
    .join(self.train_score)
    .rename_axis(index={variable:'variable'})
    .join(self.agg)
    # .join(self.mlt.query("variable==@variable").droplevel('variable').rename_axis(index={'value':variable}))
)
# S['predict'] *= S['mlt']
S.disp(1000)
# self.agg

In [ ]:
self.y_pred.reset_index().dtypes.head(10)

In [ ]:
# set(crse_codes).difference(set(self.y_true.reset_index()['variable']))
intersection(crse_codes, self.y_true.reset_index()['variable'], sort=True)


In [ ]:
from LiveAMP import *
qry = f"""
select distinct
    ssbsect_term_code, ssbsect_crn
from ssbsect
where ssbsect_subj_code = 'AGEC' and ssbsect_crse_numb = 2317
"""
qry = f"""
select
    lower(B.ssbsect_subj_code) || B.ssbsect_crse_numb as crse_code,
    A.sfrstcr_term_code as term_code,
    A.sfrstcr_pidm as pidm,
    (select C.sgbstdn_levl_code from sgbstdn C where C.sgbstdn_pidm = A.sfrstcr_pidm and C.sgbstdn_term_code_eff <= A.sfrstcr_term_code order by C.sgbstdn_term_code_eff desc fetch first 1 rows only) as levl_code,
    (select C.sgbstdn_styp_code from sgbstdn C where C.sgbstdn_pidm = A.sfrstcr_pidm and C.sgbstdn_term_code_eff <= A.sfrstcr_term_code order by C.sgbstdn_term_code_eff desc fetch first 1 rows only) as styp_code,
    sum(A.sfrstcr_credit_hr) as credit_hr
from sfrstcr A, ssbsect B
where
    A.sfrstcr_term_code = B.ssbsect_term_code
    and A.sfrstcr_term_code in (202108,202208,202308,202408)
    and A.sfrstcr_crn = B.ssbsect_crn
    and A.sfrstcr_ptrm_code not in ('28','R3')
    and B.ssbsect_subj_code <> 'INST'
group by B.ssbsect_subj_code, B.ssbsect_crse_numb, A.sfrstcr_term_code, A.sfrstcr_pidm
"""

qry = f"""
select A.crse_code, A.term_code, A.styp_code, count(*) as ct
from {subqry(qry)} A
where A.levl_code = 'UG' and A.styp_code in ('N','R','T') and A.credit_hr > 0
group by A.crse_code, A.term_code, A.styp_code
--order by A.crse_code, A.term_code, A.styp_code
"""

qry = f"""
select distinct A.crse_code--, A.styp_code
from {subqry(qry)} A
where A.ct >= 5
order by A.crse_code--, A.styp_code
"""
X = db.execute(qry)

In [ ]:
X['crse_code']#.squeeze().values#.to_list()#.disp(100)

In [ ]:
set(crse_codes).difference(X.squeeze())

In [ ]:
X.groupby(['crse_code','term_code','levl_code','styp_code']).size().reset_index().disp(2000)

In [ ]:
self.Z['actual']#.vc('variable').disp(1000)

In [ ]:
self.y_pred.vc('term_code')

In [ ]:
print(self.X.query(f"styp_code==@self.styp_code").shape, self.X_proc.shape, self.y_pred.shape)

In [ ]:
i = self.X
i.ge
# i.get_level_values('term_code')
# ('term_code')

In [ ]:
self.y_pred.groupby('term_code').groups.keys()#.vc('train_code')
self.y_pred.index.unique('term_code')

In [ ]:
Z = (
    self.X_proc
    .join(self.y_true.loc[self.crse_code])
    .fillna({c:False for c in self.y_true.columns})
    .sort_values(['actual','act_equiv_missing_','pidm'], ascending=False)
    .groupby(['term_code','sim']).filter(lambda x: x['actual'].sum() >= 5)
    .assign(mask = lambda x: x.groupby(['term_code','sim']).cumcount() % 5 > 0)
)
Z.groupby(['term_code','sim'])['actual'].sum()

In [ ]:
z = self.X.join(self.y_pred)#.groupby(['crse_code','styp_code'], dropna=False).size()
z
# z.vc('crse_code')

In [ ]:
self.y_pred
# self.X_proc

In [ ]:
Z = self.X.join(self.y_pred)#, how='inner')
Z

In [ ]:
self.y_pred

In [ ]:
from LiveAMP import *
self = AMP(cycle_day=98, param=param_lst[0])
self.get_X()
self.get_terms()
idx = ['term_code','pidm']
attr = ['id','levl_code','styp_code','admit','enroll','matric']
crse = {'crse_code':'variable','credit_hr':'value'}
X = self.X.copy()
Y = {key:
    pd.concat([T.reg for term_code, T in dct.items()])
    .assign(credit_hr=lambda y: (y['crse_code']=='_allcrse')*(y['credit_hr']) + (y['crse_code']!='_allcrse')*(y['credit_hr']>0))
    .sort_values(['crse_code',*idx])
    .set_index(idx)
    # .replace('_anycrse','_'+key)
    for key, dct in self.terms.items()}

courses = sorted(pd.concat(Y.values())['crse_code'].unique())
courses.remove('_allcrse')
# z = pd.concat([
#     X['current'][[]].assign(variable='admitted', value=True),
#     *[y.query("crse_code=='_allcrse' & credit_hr>0")[[]].assign(variable=key, value=True) for key,y in Y.items()],
# ])
# z
# for key, y in Y.items():
#     X = {j: x.join(y.query("crse_code=='_allcrse'")['credit_hr'].rename(key)) for j,x in X.items()}
#     # X = {j: x.join(y.query("crse_code=='_anycrse'")['credit_hr'].rename(key).prep(bool=True)) for j,x in X.items()}
# X['final'] = X['final'].query("final>0")
# a, b = 'current', 'final'
# for i in range(2):
#     Y[a] = pd.concat([Y[a], Y[b].query("crse_code.str.contains(@b)")])
#     a,b = b,a

# Y['current'].query("crse_code=='_current'")

# def f(*L):
#     C = [z.filter(crse).rename(columns=crse) for z in L]
#     F = [z[z.columns.difference(crse)].melt(ignore_index=False) for z in L]    
#     Z = pd.concat(F+C).prep().groupby([*idx,'variable'], sort=False).first()
#     mask = Z.eval("variable in @attr")
#     # return Z[mask].unstack().droplevel(0,1).prep(bool=True).set_index(attr, append=True).join(Z[~mask]).query("levl_code=='ug' & styp_code in ('n','r','t')")
#     return Z[mask].unstack().droplevel(0,1).prep(bool=True).set_index(attr, append=True).join(Z).query("levl_code=='ug' & styp_code in ('n','r','t')")
# Z = {'current': f(X['current'],Y['current']), 'final': f(Y['final'],X['final']).query("final>0")}
# Z['final'] = Z['final'].query("final>0")
# X = {key: z.query("variable not in @courses").unstack().droplevel(0,1).prep(bool=True) for key, z in Z.items()}
# Y = {key: z.query("variable in @courses").prep(bool=True) for key, z in Z.items()}

In [ ]:
from LiveAMP import *
self = AMP(cycle_day=98, param=param_lst[0])
self.get_X()
self.get_terms()

idx = ['term_code','pidm']
X = self.X.copy()
Y = {key:
    pd.concat([T.reg for term_code, T in dct.items()])
    .assign(credit_hr=lambda y: (y['crse_code']=='_allcrse')*(y['credit_hr']) + (y['crse_code']!='_allcrse')*(y['credit_hr']>0))
    .sort_values(['crse_code',*idx])
    .set_index(idx)
    # .replace('_anycrse','_'+key)
    for key, dct in self.terms.items()}

courses = sorted(pd.concat(Y.values())['crse_code'].unique())
courses.remove('_allcrse')
attr = ['id','levl_code','styp_code','admit','enroll','matric','variable']
crse = {'crse_code':'variable','credit_hr':'value'}
def g(key):
    qry = "crse_code=='_anycrse'"
    h = lambda D: D[key].drop(columns=crse, errors='ignore').melt(ignore_index=False)
    Z = pd.concat([
        X['current']           [[]].assign(variable='admit' , value=True),
        Y['current'].query(qry)[[]].assign(variable='enroll', value=True),
        Y['final'  ].query(qry)[[]].assign(variable='matric', value=True),
        # Y[key].filter(crse).rename(columns=crse),
        Y[key][crse.keys()].rename(columns=crse),
        h(Y),
        h(X),
    ]).dropna().astype('string').groupby([*idx,'variable'], sort=False).first()
    mask = Z.eval("variable in @attr")
    return (
        Z[mask].unstack().droplevel(0,1).prep(bool=True)
        .join(Z[~mask])
        .query("levl_code=='ug' & styp_code in ('n','r','t')")
        .reset_index()
        .assign(**{k: lambda z: encrypt(z[k]) for k in ['pidm','id']})
        .set_index(idx+attr)
    )
Z = {'current':g('current').query('admit'), 'final':g('final').query('matric')}

g = lambda df: df.groupby(['term_code','levl_code','styp_code','variable','value']).size()
dct = {
    'admit' : g(Z['current']),
    'enroll': g(Z['current'].query('enroll')),
    'matric': g(Z['final']),
}
dct['mlt'] = dct['matric'] / g(Z['final'].query('admit'))
agg = pd.DataFrame(dct)

In [ ]:
Y['final'].reset_index().query("(credit_hr.isnull() or credit_hr==0) and crse_code not in ['_allcrse','_anycrse']").sort_values(['term_code','crse_code']).disp(100)

In [ ]:
X = self.X.query(f"styp_code==@self.styp_code").rename_axis(columns=None)#.rename(columns=lambda x:'__'+x)

# trf = ColumnTransformer(self.param['trf'][2], remainder='drop', verbose_feature_names_out=False)
# {x[0] for x in self.param['trf'][2]}.difference(X.columns)
# # trf
# X_trf = trf.fit_transform(X)#.prep(bool=True, cat=True)
# X.rename_axis(columns='')


In [ ]:
crse_code = 'math1314'
self.X.join(self.Y.loc[crse_code]).fillna({c:False for c in self.Y.columns})

In [ ]:
from LiveAMP import *
self = AMP(cycle_day=98, param=param_lst[0])
self.get_X()
self.get_terms()

idx = ['term_code','pidm']
crse = {'crse_code':'variable','credit_hr':'value'}
X = self.X.copy()


Y = {key:
    pd.concat([T.reg for term_code, T in dct.items()])
    .dropna()
    .sort_values(['crse_code',*idx])
    .set_index(idx)
    .assign(credit_hr=lambda y: (y['crse_code']=='_allcrse')*(y['credit_hr']) + (y['crse_code']!='_allcrse')*(y['credit_hr']>0))
    for key, dct in self.terms.items()}

idx = ['variable','term_code','pidm']
attr = ['id','levl_code','styp_code','admit','enroll','matric']
crse = {'crse_code':'variable','credit_hr':'value'}
def g(key):
    qry = "crse_code=='_anycrse'"
    h = lambda D: D[key].drop(columns=crse, errors='ignore').melt(ignore_index=False)
    Z = pd.concat([
        X['current']           [[]].assign(variable='admit' , value=True),
        Y['current'].query(qry)[[]].assign(variable='enroll', value=True),
        Y['final'  ].query(qry)[[]].assign(variable='matric', value=True),
        Y[key].filter(crse).rename(columns=crse),
        h(Y),
        h(X),
    ]).dropna().astype('string').groupby(idx, sort=False).first()
    mask = Z.eval("variable in @attr")
    return (
        Z[mask].unstack(0).droplevel(0,1).prep(bool=True)
        .join(Z[~mask])
        .query("levl_code=='ug' & styp_code in ('n','r','t')")
        .reset_index()
        .assign(pidm=lambda z: encrypt(z['pidm']), id=lambda z: encrypt(z['id']))
        .set_index(idx+attr)
    #     # .set_index(['variable',*idx,*attr])
    )
self.Z = {'current':g('current').query('admit'), 'final':g('final').query('matric')}
# self.Z = {'current':g('current'), 'final':g('final')}


In [ ]:
Y['final'].filter(crse).rename(columns=crse)

In [ ]:
self.Z['final'].reset_index().missing()
# self.Z['current'].query("id.isnull()")

In [ ]:
from LiveAMP import *
self = AMP(cycle_day=98, param=param_lst[0])
self.get_X()
self.get_terms()

idx = ['term_code','pidm']
crse = {'crse_code':'variable','credit_hr':'value'}
X = self.X.copy()


Y = {key:
    pd.concat([T.reg for term_code, T in dct.items()])
    .sort_values(['crse_code',*idx])
    .set_index(idx)
    .assign(credit_hr=lambda y: (y['crse_code']=='_allcrse')*(y['credit_hr']) + (y['crse_code']!='_allcrse')*(y['credit_hr']>0))
    for key, dct in self.terms.items()}

courses = sorted(pd.concat(Y.values())['crse_code'].unique())
courses.remove('_allcrse')
idx = ['variable','term_code','pidm']
attr = ['id','levl_code','styp_code','admit','enroll','matric']
def g(key):
    qry = "crse_code=='_anycrse'"
    h = lambda D: D[key].drop(columns=crse, errors='ignore').melt(ignore_index=False)
    Z = pd.concat([
        X['current']           [[]].assign(variable='admit' , value=True),
        Y['current'].query(qry)[[]].assign(variable='enroll', value=True),
        Y['final'  ].query(qry)[[]].assign(variable='matric', value=True),
        Y[key].filter(crse).rename(columns=crse),
        h(Y),
        h(X),
    ]).dropna().astype('string').groupby(idx, sort=False).first()
    mask = Z.eval("variable in @attr")
    return (
        Z[mask].unstack(0).droplevel(0,1).prep(bool=True)
        .join(Z[~mask])
        .query("levl_code=='ug' & styp_code in ('n','r','t')")
        .reset_index()
        .assign(**{k: lambda z: encrypt(z[k]) for k in ['pidm','id']})
        .set_index(idx+attr)
        # .set_index(['variable',*idx,*attr])
    )
Z = {'current':g('current').query('admit'), 'final':g('final').query('matric')}
X = Z['current'].query("variable not in @courses").unstack(0).droplevel(0,1).prep(bool=True)
Y = [z.query("variable in @courses").droplevel(attr).rename(columns={'value':key}) for key,z in Z.items()]
Y = Y[0].join(Y[1], how='outer').prep(bool=True)

g = lambda df: df.groupby(['term_code','levl_code','styp_code','variable','value']).size()
dct = {
    'admit' : g(Z['current']),
    'enroll': g(Z['current'].query('enroll')),
    'matric': g(Z['final']),
}
dct['mlt'] = dct['matric'] / g(Z['final'].query('admit'))
agg = pd.DataFrame(dct)


# crse_code = 'math1314'
# for key, z in Z.items():
#     X[key] = z.query("variable==@crse_code").droplevel(attr).prep(bool=True)


In [ ]:
Y.loc['math1314']

In [ ]:
X['final'].drop(columns='value', errors='ignore').melt(ignore_index=False)#.melt(id_vars=idx)

In [ ]:
Y = [z.query("variable in @courses").droplevel(attr).rename(columns={'value':key}) for key,z in Z.items()]
Y = Y[0].join(Y[1], how='outer').prep(bool=True)
# pd.concat(Y, axis=1)

In [ ]:
crse_code = 'math1314'
Y.loc[crse_code]

In [ ]:
Y.loc[:,:,'math1314']

In [ ]:
crse_code = 'math1314'
for key, z in Z.items():
    z.query("variable==@crse_code").rsindex(idx).prep(bool=True).disp()


In [ ]:
Z['final'].query("variable==@crse_code").droplevel(attr).prep()#.sum()

In [ ]:
X = Z['current'].query("variable not in @courses").unstack().droplevel(0,1).prep(bool=True)
crse_code='math1314'
print(X.shape)
for key, z in Z.items():
    X[key] = z.query("variable==@crse_code").droplevel(attr).prep(bool=True)
    # X[key] = z.query("variable==@crse_code").droplevel(attr).astype('boolean')
# y = [z.query("variable==@crse_code").droplevel(attr).rename(columns={'value':key}) for key,z in Z.items()]
# y[0]
print(X.shape)
X.iloc[:,-2:].sum()

In [ ]:
X.dtypes

In [ ]:
X.disp()

In [ ]:
X['current'].sum()
X.dtypes

In [ ]:
X = Z['current'].query("variable not in @courses").unstack().droplevel(0,1).prep(bool=True)
y = [z.query("variable in @courses").droplevel(attr).rename(columns={'value':key}) for key,z in Z.items()]
y = y[0].join(y[1], how='outer').prep(bool=True)
y

In [ ]:
Z['final'].groupby(['term_code','levl_code','styp_code','variable','value'])

In [ ]:
z = Z['final']
mask = z.isnull().any(axis=1)
z[mask]

In [ ]:
# y = [z.droplevel(attr).query("variable in @courses") for key,z in Z.items()]
y = [z.droplevel(attr).query("variable in @courses").rename(columns={'value':key}) for key,z in Z.items()]
y[0]
# pd.concat(y, axis=1)

In [ ]:
X = Z['current'].query("variable not in @courses").unstack().droplevel(0,1).prep(bool=True)
y = [z.query("variable in @courses").droplevel(attr).rename(columns={'value':key}) for key,z in Z.items()]

# y = [z.droplevel(attr).query("variable in @courses").rename(columns={'value':key}) for key,z in Z.items()]
# y = y[0].join(y[1], how='outer').prep(bool=True)
# pd.concat(y, axis=1)


In [ ]:
y

In [ ]:
crse_code = 'math1314'
X.join(y.query(f"variable==@crse_code"))

In [ ]:
y

In [ ]:
crse_code = 'math1314'
X.join(y.query("variable==@crse_code")).droplevel(['id','enroll'])


In [ ]:
#.sum(axis=1).value_counts()

# y[0].to_frame().join(y[1], how='outer')#.sum(axis=1).value_counts()


# y = {key: z.query("variable in @courses").squeeze().prep().rename(key) for key,z in Z.items()}
# pd.DataFrame(y)
# y['final'].reset_index().min()

In [ ]:
y = {key: z.query("variable in @courses").rename(columns={'value':key}).prep() for key,z in Z.items()}
# pd.concat(y.values(), axis=1)
y['current'].join(y['final']).sum(axis=1).value_counts()
# y = pd.concat([Z[key].query("variable in @courses").
# y = pd.DataFrame({key: z.query("variable in @courses").squeeze() for key,z in Z.items()})
# y

In [ ]:
X

In [ ]:
dct['mlt'].query("variable=='_allcrse'")

In [ ]:
Z['current'].query("variable=='act_equiv'")

In [ ]:
Z = pd.concat([
    X['current']['apdc_day'].rename('admitted'),
    *[y.query("crse_code=='_allcrse'")['credit_hr'].rename(key) for key,y in Y.items()]
    ])
Z

In [ ]:
Z = (pd.concat([
    X['current']['apdc_day'].rename('admitted'),
    *[y.query("crse_code=='_allcrse'")['credit_hr'].rename(key) for key,y in Y.items()]
    ], axis=1) > 0).fillna(False)
mask = Z.sum(axis=1) == 0
Z[mask]
Z.melt(ignore_index=False)

In [ ]:
# Z = (
#     X['current'][[]].assign(current=True)
#     .join(Y['final'].query("crse_code=='_allcrse' & credit_hr>0")[[]].assign(final=True), how='outer')
#     .prep(bool=True).fillna(False)
#     .assign(yld=lambda z: z.all(axis=1))
# )
# Z['yield'] = Z['current'] & Z['final']
qry = "crse_code=='_allcrse' & credit_hr>0"
Z = pd.concat([
    X['current']           [[]].assign(admitted=True),
    Y['current'].query(qry)[[]].assign(enrolled=True),
    Y['final'  ].query(qry)[[]].assign(final   =True),
    ], axis=1).prep(bool=True).fillna(False)
# A = X['current'][].assign(current=True).join(Y['final'].query("crse_code=='_allcrse' & credit_hr>0")[[]].assign(final=True), how='outer').prep(bool=True).fillna(False)

                                           
                                           
                                        #    .query([[]].assign(current=True))
  

In [ ]:
# Y['final'].query("crse_code=='_allcrse' & credit_hr>0")
Y['final'].query("pidm.notnull()")

In [ ]:
X['final']

In [ ]:
A = X['current'][[]]#.assign(a=True)
B = X['final'][[]]#.assign(b=True)
A.join(B, how='outer')

In [ ]:
A = pd.concat([x[[]].assign(**{key:True}).prep() for key,x in X.items()], axis=1).fillna(False)
A#.query('pidm==2376')

In [ ]:
Z['current'].query("variable not in @courses")


In [ ]:
A.query("variable=='_anycrse'").disp(30)
B.query("variable=='_anycrse'").disp(30)
C.query("variable=='_anycrse'").disp(30)

In [ ]:
agg = lambda df, nm: df.groupby(['term_code','levl_code','styp_code','variable','value']).size().squeeze().rename(nm)
A = agg(Z['current']                 , 'current')
B = agg(Z['current'].query("final>0"), 'yield')
C = agg(Z['final'  ].query("final>0"), 'final')
M = pd.concat([A,B,C], axis=1)
M['mlt'] = M['final'] / M['yield']
M.query("variable=='_anycrse'")
# C.query("variable=='_anycrse'").disp(30)

# M = {key: z.groupby(['term_code','levl_code','styp_code','variable','value']).size() for key, z in Z.items()}
# M = {key: z.groupby(['term_code','levl_code','styp_code','variable','value']).size() for key, z in Z.items()}
# M['current']
# Z['current']

In [ ]:
M = pd.concat([A,B,C], axis=1)
M['mlt'] = M['final'] / M['yield']
M.query("variable=='_anycrse'")

In [ ]:
Z['current'].reset_index().dtypes

In [ ]:
# Q = {key: z.query("variable not in @courses") for key, z in Z.items()}
Z['final']

In [ ]:
Z['final'].query("id.isnull() & variable=='styp_code'")#.vc('styp_code')

In [ ]:
M = {key: z.groupby(['term_code','levl_code','styp_code','variable','value']).size() for key, z in Z.items()}

In [ ]:
# Z['current'].query("variable=='styp_code'").groupby(['term_code','levl_code','styp_code','variable','value']).size().query("variable=='styp_code'")
Z['current'].groupby(['term_code','levl_code','styp_code','variable','value']).size().query("variable=='styp_code'")

In [ ]:
# mlt = M['final'] / M['current']
# mlt.query("variable=='_anycrse'")
# M['current'].vc('variable')#.query("variable=='_anycrse'")
M['current'].query("variable=='styp_code'")

In [ ]:
courses[:10]

In [ ]:
Y = {key: z.query("variable in @courses").prep(bool=True) for key, z in Z.items()}
Y['final']#.reset_index().dtypes

In [ ]:
X = {key: z.query("variable not in @courses").unstack().droplevel(0,1).prep(bool=True) for key, z in Z.items()}
X['final'].reset_index().dtypes

In [ ]:
Z['final'].query("variable not in @courses").unstack().droplevel(0,1)

In [ ]:
courses = pd.concat(Y.values())['crse_code'].unique().tolist()
courses

In [ ]:
a = Z['current']
a
mask = a.eval("variable in @attr")
c = a[mask].unstack().droplevel(0,1).set_index(attr, append=True).join(a[~mask])
c
# a[~mask].join(b)

# .squeeze().rename('styp_code').droplevel('variable')
# a.join(b)
# pd.concat([a,], axis=1)

In [ ]:
X['final']

In [ ]:
from LiveAMP import *
# Flags().run()
self = AMP(cycle_day=98, param=param_lst[0])
self.get_X()
self.get_terms()

X = self.X.copy()
idx  = ['term_code','pidm']
Y = {key: pd.concat([
    T.reg
    # .fillna({'credit_hr':0})
    # .query('credit_hr>0')
    .assign(pidm=lambda x: encrypt(x['pidm']))

    # .assign(pidm=lambda x: encrypt(x['pidm']), variable='crse_code')#, value=lambda x: x['crse_code'])
    .set_index(idx)
    # .rename(columns={'crse_code':'level', 'credit_hr':'value'})
    # .prep(bool=True)
    
#     .filter(col)
for term_code, T in dct.items()]) for key, dct in self.terms.items()}

# E = {key: y.query("value=='_allcrse'")[attr] for key, y in Y.items()}
# E['final']
# # A = pd.concat(E.values(), axis=1, join='outer')
# A = E['current'].join(E['final'], how='outer', lsuffix='_x', rsuffix='_y')
# mask = A['levl_code_x'] != A['levl_code_y']
# mask = A['styp_code_x'] != A['styp_code_y']
# A[mask]
# # A.query()
# Y['final'].droplevel(attr).join(X['final'], how='outer')
Y['final']
col = ['crse_code','credit_hr']
a = Y['final'].drop(columns=col)
a

In [ ]:
C.sort_values('variable')
courses = C['variable'].unique().tolist()
courses

In [ ]:
col = ['crse_code','credit_hr']
C = Y['final'].filter(col).sort_values('crse_code').rename(columns={'crse_code':'variable','credit_hr':'value'})
A = Y['final'].drop(columns=col).melt(ignore_index=False)
B = X['final'].set_index(['term_code','pidm'])[['id','levl_code','styp_code','distance','international']].melt(ignore_index=False)
D = pd.concat([A,B,C]).groupby([*idx,'variable'], sort=False).first()
D.query("variable in @courses")
D.query("variable not in @courses").unstack().droplevel(0,1)

In [ ]:
D.unstack(sort=False).prep(bool=True).disp()

In [ ]:
from LiveAMP import *
# Flags().run()
self = AMP(cycle_day=98, param=param_lst[0])
self.get_X()
self.get_terms()
# self = run_amp(98)

idx = ['term_code','pidm','id']
attr = ['levl_code','styp_code']
col = ['variable','value',*idx,*attr,'current','final']
# col = attr + ['variable','value','current','final']
X = self.X.copy()
Y = {key: pd.concat([
    T.reg
    # .fillna({'credit_hr':0})
    # .query('credit_hr>0')
    .rename(columns={'crse_code':'value', 'credit_hr':key})
    .assign(pidm=lambda x: encrypt(x['pidm']), variable='crse_code')#, value=lambda x: x['crse_code'])
    .prep(bool=True)
    .sindex(idx)
    .filter(col)
for term_code, T in dct.items()]) for key, dct in self.terms.items()}


f = lambda df: (df.query("value=='_allcrse'") if 'value' in df else df).filter(col)
A = (
    f(X['current'])
    .join(f(X['final'  ]), how='outer', lsuffix='currentX', rsuffix='finalX')
    .join(f(Y['current']), how='outer')
    .join(f(Y['final'  ]), how='outer', lsuffix='currentY', rsuffix='finalY')
    .fillna({'current':0,'final':0})
)
g = lambda a, b: f(A.assign(**{key: A[key+a].combine_first(A[key+b]) for key in attr}))#.set_index(attr, append=True)
# g = lambda a, b: f(Z.assign(levl_code=Z['levl_code'+a].combine_first(Z['levl_code'+b]), styp_code=Z['styp_code'+a].combine_first(Z['styp_code'+b])))
Z = {'current': g('currentX', 'currentY'), 'final':g('finalY', 'finalX')}
h = lambda dct: {key: dct[key].drop(columns=z.columns, errors='ignore').join(z).set_index(attr, append=True) for key, z in Z.items()}
X = h(X)
Y = h(Y)
Z['current']
A
# qry = f"levl_code == 'ug' & styp_code in ('n','r','t')"
# # grp = ['variable','value','term_code'] + attr
# col.remove('pidm')
# col.remove('id')
# col
# aggy = lambda y: y.query(qry).groupby(col).size()
# aggy(Y['current'])

In [ ]:
X = self.X.copy()
f = lambda df: (df.query("value=='_allcrse'") if 'value' in df else df).filter(attr+val)
A = (
    f(X['current'])
    .join(f(X['final'  ]), how='outer', lsuffix='currentX', rsuffix='finalX')
    .join(f(Y['current']), how='outer')
    .join(f(Y['final'  ]), how='outer', lsuffix='currentY', rsuffix='finalY')
)
g = lambda a, b: f(A.assign(**{key: A[key+a].combine_first(A[key+b]) for key in attr}))#.set_index(attr, append=True)
# g = lambda a, b: f(Z.assign(levl_code=Z['levl_code'+a].combine_first(Z['levl_code'+b]), styp_code=Z['styp_code'+a].combine_first(Z['styp_code'+b])))
Z = {'current': g('currentX', 'currentY'), 'final':g('finalY', 'finalX')}
h = lambda dct: {key: dct[key].drop(columns=z.columns, errors='ignore').join(z).set_index(attr, append=True) for key, z in Z.items()}
X = h(X)
Y = h(Y)
# X['current']
Z['current']

In [ ]:
Z['current']

In [ ]:
Y['current']

In [ ]:
def g(A,B):
    C = A.join(B, how='outer', rsuffix='_x')
    for key in attr:
        C[key] = C[key].combine_first(C[key+'_x'])
        C = C.drop(columns=key+'_x')
    return C
g(E['current'], E['final'])

# Z = pd.concat([e for key, e in E.items()], axis=1)
# E['current'].join(E['final'], how='outer', rsuffix='_x')

# Z

# E['current']

In [ ]:
E['final']

In [ ]:
def g(A,B):
    C = A.join(B, rsuffix='_x')
    for key in attr:
        C[key] = C[key].combine_first(C[key+'_x'])
        C = C.drop(columns=key+'_x')
    return C


A = X['current']
B = E['current'][[]]
g(A,B)
# C = A.join(B, rsuffix='_x')
# for key in attr:
#     C[key] = C[key].combine_first(C[key+'_x'])
#     C = C.drop(columns=key+'_x')
# # C = C.drop(columns=[key])
# C

In [ ]:
X['final']

In [ ]:
Y['current']

In [ ]:
Z = Y['current'].query("value=='_allcrse'")[['current']].join(Y['final'].query("value=='_allcrse'")[['final']], how='outer')
Z.notnull().sum(axis=1).query('pidm==9568241')#.describe()
# Z.query('pidm==9568241').disp(100)

In [ ]:
ids

In [ ]:
X = self.X.copy()
XX = pd.concat(X.values())
idx = ['cycle_day','term_code','pidm','id']
XXX = XX.groupby(idx).size()
XXX[XXX>0]#.vc('term_code')
XXX.vc('cycle_day')
# XX.query(f"term_code==202408 & pidm==9175235").drop_duplicates().disp(5)

In [ ]:
ids = pd.concat(X.values())[idx].drop_duplicates()#.groupby(idx).first()
ids

In [ ]:
Y = {key: pd.concat([
    T.reg
    .assign(pidm=lambda x: encrypt(x['pidm']), variable='crse_code', value=lambda x: x['crse_code'])
    .fillna({'credit_hr':0})
    .query('credit_hr>0')
    .merge(ids, how='left')
    .prep(bool=True)
    # .set_index(idx)
    .rename(columns={'credit_hr':key})
    [[*idx,*attr,'variable','value',key]]
    # [[*idx,key]]
#     # .sindex(idx)
#     # ['credit_hr']
#     # .rename(key)
for term_code, T in dct.items()]) for key, dct in self.terms.items()}
Y['final']
# X['current']
# [idx].drop_duplicates()
# idx += col
# X

In [ ]:
X = self.X.copy()
idx = ['term_code','pidm','id']
col = ['levl_code','styp_code']
ids = pd.concat(X.values())[idx].drop_duplicates()
idx += col
idx = ['variable','value',*idx,*col]
Y = {key: pd.concat([
    T.reg
    .assign(pidm=lambda x: encrypt(x['pidm']), variable='crse_code', value=lambda x: x['crse_code'])
    .fillna({'credit_hr':0})
    .query('credit_hr>0')
    .merge(ids, how='left')
    .prep(bool=True)
    .set_index(idx)
    .rename(columns={'credit_hr':key})
    [[key]]
    # [[*idx,key]]
#     # .sindex(idx)
#     # ['credit_hr']
#     # .rename(key)
for term_code, T in dct.items()]) for key, dct in self.terms.items()}
z = Y['current'].droplevel(col).join(Y['final'].droplevel(col))

def g(A, B):
    C = A.reset_index().merge(B.reset_index(), on=idx, how='outer', suffixes=('','_x'))
    for key in col:
        C[key] = C[key].combine_first(C[key+'_x'])
    C = C.groupby(idx+col).first()[[]]
    return A.droplevel(col).join(C), B.droplevel(col).join(C)
X['current'], Y['current'] = g(X['current'], Y['current'])
Y['final'  ], X['final'  ] = g(Y['final'  ], X['final'  ])

for key in ['current','final']:
    X[key] = X[key].join(z)

In [ ]:
X['current']

In [ ]:
z

In [ ]:
X['current'].query('term_code==202408')
X['final'].query('term_code==202408')

In [ ]:
mask = z['current'] != z['final']
z[mask]

In [ ]:
# X, Y = self.X.copy(), self.Y.copy()
# z = pd.concat([y.query(f"value=='_allcrse'").rsindex(['pidm','id','term_code']) for key,y in Y.items()], axis=1)
def g(A, B):
    idx = ['pidm','id','term_code']
    col = ['levl_code','styp_code']
    C = A.reset_index().merge(B.reset_index(), on=idx, how='outer', suffixes=('','_x'))
    for key in col:
        C[key] = C[key].combine_first(C[key+'_x'])
    C = C.groupby(idx+col).first()[[]]
    return A.droplevel(col).join(C), B.droplevel(col).join(C)
X['current'], Y['current'] = g(X['current'], Y['current'])
Y['final'  ], X['final'  ] = g(Y['final'  ], X['final'  ])

for key in ['current','final']:
    X[key] = X[key].join(z)

In [ ]:
X['final']

In [ ]:
z

In [ ]:
X['final']

In [ ]:
Y['final']

In [ ]:
Q = Y['final'].groupby(['value','pidm','id','term_code']).size()
Q[Q>1]

In [ ]:
# Z = drp(E['current']).join(drp(E['final']), how='outer')
# {key: y.query(f"value=='_allcrse'").rename(columns={'credit_hr':key}) for key,y in Y.items()}
z = pd.concat([y.query(f"value=='_allcrse'").rsindex(['pidm','id','term_code']).rename(columns={'credit_hr':key}) for key,y in Y.items()], axis=1)

#     # .groupby(['pidm','id','term_code'], dropna=False)
#     # .sum()
#     # .rsindex(['pidm','id'])
#     # .squeeze()
#     # .rename(key)
# for key,y in Y.items()], axis=1)
z
# z[0]
# pd.concat(z, axis=1)

In [ ]:
z[0]

In [ ]:
Y['current']

In [ ]:
for key in ['current','final']:
    X[key] = X[key].join(Y['current'])
X['']

In [ ]:
Y['final']

In [ ]:
X, Y = self.X.copy(), self.Y.copy()
def g(A, B):
    # g = lambda df: df.reset_index().set_index(['pidm','id','term_code'])
    idx = ['pidm','id','term_code']
    col = ['levl_code','styp_code']
    C = A.reset_index().set_index(idx)[col].join(B.reset_index().set_index(idx)[col], how='outer', rsuffix='_x')
    # C = A[[]].reset_index(col).rsindex().join(B[[]].reset_index(col).rsindex(), how='outer', rsuffix='_x')
    for key in col:
        C[key] = C[key].combine_first(C[key+'_x'])
    C[col].disp(3)
    C = C.set_index(col, append=True)[[]]
    return A.droplevel(col).join(C), B.droplevel(col).join(C)
g(X['current'], Y['current'])
# C = A.merge(B, on=['pidm','id','term_code'], how='outer', suffixes=('','_x'))
# C['levl_code'] = C['levl_code'].combine_first(C['levl_code_x'])
# C['styp_code'] = C['styp_code'].combine_first(C['styp_code_x'])
# C.drop(['levl_code_y','styp_code_y'])

In [ ]:
X, Y = self.X.copy(), self.Y.copy()
idx = ['variable','value','id','pidm','levl_code','styp_code','term_code']
qry = f"levl_code == 'ug' & styp_code in ('n','r','t')"
E = {key: y.query("value=='_allcrse'").droplevel(['variable','value']).rename(columns={'credit_hr':key}) for key, y in Y.items()}  # only rows for _allcrse
drp = lambda df: df.droplevel(['levl_code','styp_code'])
E['current'] = drp(E['current']).join(X['current'][[]], how='outer')  # X determines current levl_code & styp_code
X['final'  ] = drp(X['final'  ]).join(E['final'  ][[]], how='outer')  # E determines final   levl_code & styp_code
Z = drp(E['current']).join(drp(E['final']), how='outer')
for key in ['final','current']:
    X[key] = X[key].join(Z)  # join current & final credit hours
Y = {key: drp(y[[]]).join(E[key][[]]) for key, y in Y.items()}  # copy levl_code & stype_code from E to Y

# E['current'] = E['current'].droplevel(col).join(X['current'][[]], how='outer')  # X determines current levl_code & styp_code
# X['final'  ] = X['final'  ].droplevel(col).join(E['final'  ][[]], how='outer')  # E determines final   levl_code & styp_code
# z = E['current'].droplevel(col).join(E['final'].droplevel(col), how='outer')

# for key in ['final','current']:
#     X[key] = X[key].join(E['current'].droplevel(col)).join(E['final'].droplevel(col))  # join current & final credit hours
# Y = {key: y[[]].droplevel(col).join(E[key][[]]) for key, y in Y.items()}  # copy levl_code & stype_code from E to Y
cols = sorted(c for c in X['final'].select_dtypes(['string','boolean']).columns if "_missing" not in c and "_code" not in c)
idx.remove('id')
idx.remove('pidm')
aggy = lambda y: y.query(qry).groupby(idx).size()
# aggx = lambda x: aggy(x[cols].melt(ignore_index=False))
# def get_df(dct):
#     Y = pd.concat(dct, axis=1).prep().fillna(0)
#     Y['mlt'] = Y['final'] / Y['admitted']
#     Y[np.isinf(Y)] = pd.NA
#     return Y#.rename(columns=lambda x: "term_code_"+x)

AY = get_df({
    'current': aggy(Y['current']),
    'admitted': aggy(Y['current'][[]].join(drp(Y['final']), how='inner')),
    'final': aggy(Y['final']),
})
# AX = get_df({
#     'current': aggx(X['current'].query("current.notnull()")),
#     'admitted': aggx(X['current'].query("final.notnull()")),
#     'final': aggx(X['final'].query("final.notnull()")),
# })

# AX.query(f"value in ('_allcrse','biol1406') & term_code==202408")


In [ ]:
aggy(X['current'][[]].join(drp(Y['final']), how='inner'))

In [ ]:
aggy(Y['current'][[]].join(drp(Y['final']), how='inner'))
# drp(Y['final'])

In [ ]:
E['current'].droplevel(col).join(E['final'].droplevel(col), how='outer')

In [ ]:
Y['current']#.query("current.notnull()")['current'].min()

In [ ]:
AX

In [ ]:
qry = f"term_code==202408 & styp_code=='r' & pidm in [9207146,9240010,10313929]"
X['current'].query(qry).disp(10)
X['final'].query(qry).disp(10)
qry += " &value=='biol1406'"
Y['current'].query(qry).disp(10)
Y['final'].query(qry).disp(10)

In [ ]:


# E['final'  ] = E['final'  ].droplevel(col).join(X['final'  ][[]], how='outer')  # copy id from X to E
# E['current'] = E['current'].droplevel(col).join(X['current'][[]], how='outer')  # X determines current levl_code, styp_code, & credit_hr

# X['current'] = X['current'].join(E['current'], how='outer')  # copy current credit_hr from E to X

# X['current'] = X['current'].join(E['final'  ].droplevel(col)    , how='outer')  # copy final credit_hr from E to X
# X['final'  ] = X['final'  ].join(E['current'].droplevel(col)    , how='outer')  # copy current credit_hr from E to X

# Y = {key: y[[]].droplevel(col).join(E[key][[]]) for key, y in Y.items()}  # copy levl_code & stype_code from E to Y

# cols = sorted(c for c in X['final'].select_dtypes(['string','boolean']).columns if "_missing" not in c and "_code" not in c)
# idx.remove('id')
# idx.remove('pidm')
# aggy = lambda y: y.query(qry).groupby(idx).size()
# aggx = lambda x: aggy(x[cols].melt(ignore_index=False))
# def get_df(dct):
#     Y = pd.concat(dct, axis=1).prep().fillna(0)
#     Y['mlt'] = Y['final'] / Y['admitted']
#     Y[np.isinf(Y)] = pd.NA
#     return Y#.rename(columns=lambda x: "term_code_"+x)

# AY = get_df({
#     'current': aggy(Y['current']),
#     'admitted': aggy(X['current'][[]].join(Y['final'].droplevel(col))),
#     'final': aggy(Y['final']),
# })
# AY.query(f"value in ('_allcrse','biol1406')")

In [ ]:
Y['final'].vc('styp_code')

In [ ]:
id = pd.concat(X.values())[[]].groupby(['id','pidm']).first()
Y['current'].join(id)
# A = Y['current'].join(#.drop_duplicates().set_index('pidm'))
# A = Y['current'].join(pd.concat(X.values()).reset_index()[['id','pidm']].drop_duplicates().set_index('pidm'))
# A
# A.query('id.isnull()').vc('styp_code')
# X['final']

In [ ]:
pd.concat(X.values()).reset_index()[['id','pidm']].drop_duplicates()

In [ ]:
key = 'final'
X[key].query(qry).query("term_code==202408")#.query("pidm==9175235")
# E[key].query(qry).query("term_code==202408")
# Y[key].droplevel(col).join(E[key][[]])

In [ ]:
Y['final'].query(qry).query('term_code==202408')

In [ ]:
X, Y = self.X.copy(), self.Y.copy()
idx = ['variable','value','id','pidm','levl_code','styp_code','term_code']

E = {key, y.query("value=='_allcrse'").droplevel(['variable','value']).squeeze() for key, y in Y.items()}  # only rows for _allcrse
E[0] = E[0].rename('final').to_frame()
E[1] = E[1].rename('current').to_frame()
col = ['levl_code','styp_code']
X[0] = X[0].droplevel(col).join(E[0], how='outer')  # E[0] determines final levl_code, styp_code, & credit_hr
E[0] = E[0].droplevel(col).join(X[0][[]], how='outer')  # copy id from X[0] to E[0]
E[1] = E[1].droplevel(col).join(X[1][[]], how='outer')  # X[1] determines current levl_code & styp_code
X[1] = X[1].droplevel(col).join(E[1], how='outer')  # copy current credit_hr from E[1] to X[1]
X[1] = X[1].join(E[0].droplevel(col), how='outer')  # copy final credit_hr from E[0] to X[1]
X[0] = X[0].join(E[1].droplevel(col), how='outer')  # copy current credit_hr from E[1] to X[0]
Y = [y[[]].droplevel(col).join(e[[]]) for y,e in zip(Y,E)]  # copy levl_code & stype_code from E to Y
qry = f"levl_code == 'ug' & styp_code in ('n','r','t')"
cols = sorted(c for c in X[0].select_dtypes(['string','boolean']).columns if "_missing" not in c and "_code" not in c)
idx.remove('id')
idx.remove('pidm')
aggy = lambda y: y.query(qry).groupby(idx).size()
aggx = lambda x: aggy(x[cols].melt(ignore_index=False))
def get_df(dct):
    Y = pd.concat(dct, axis=1).prep().fillna(0)
    Y['mlt'] = Y['final'] / Y['admitted']
    Y[np.isinf(Y)] = pd.NA
    return Y#.rename(columns=lambda x: "term_code_"+x)

AY = get_df({
    'current': aggy(Y[1]),
    'admitted': aggy(X[1][[]].join(Y[0].droplevel(col))),
    'final': aggy(Y[0]),
})

In [ ]:
Y[0]
X[1]

In [ ]:
AY.query(f"value in ('_allcrse','biol1406')")

In [ ]:
self.mlt.query(f'term_code==202408')#.vc('term_code')

In [ ]:
M = self.mlt.copy().reset_index()
# M['mlt_code'] = M['term_code']
# M = pd.concat([M, M.assign(term_code=self.proj_code)]).set_index(['variable','levl_code','styp_code','term_code','mlt_code','value'])
# M.groupby(['variable','term_code','styp_code'])
M.query("value=='_allcrse'")

In [ ]:
variable = 'crse_code'
grp = [variable,'levl_code','styp_code','term_code','train_code','sim','trf_hash','imp_hash','clf_hash']
S = (Z
    .groupby(grp).apply(lambda y: pd.Series({
            'predict': y['proba'].sum(),
            'test_score': log_loss(y['actual'], y['proba'], labels=[False,True]) * 100,
        }), include_groups=False)
    .join(self.train_score)
    # .join(self.mlt.query("variable==@variable").droplevel('variable').rename_axis(index={'value':variable}))
    .reset_index()
)
S
# S['train_code'] = S['train_code'].astype('string').replace(str(self.proj_code), 'all')
# S['predict'] *= S['mlt']
# alpha = 1
# S['overall_score'] = (S['train_score'] + alpha * S['test_score']) / (1 + alpha)
# S

In [ ]:
msk = S.eval("term_code == @self.proj_code")
proj_col = f'{self.proj_code}_projection'
S['predict'] *= S['mlt']
# T = S[~msk].join(S[msk].droplevel('term_code')['predict'].rename(proj_col))

# for k in ['predict',proj_col]:
#     T[k] *= T['mlt']


    # [[proj_col,'current','admitted','final','predict','overall_score']]
# )
S.disp(100)

In [ ]:
L = [pd.concat([df.droplevel('term_code')], axis=1, keys=[term_code]) for term_code, df in A.groupby('term_code')]
df = pd.concat(L, axis=1)
df
# pd.concat([df], axis=1, keys=['C'])
# def g(df):
#     df.columns = pd.MultiIndex.from_product([['C'], df.columns, ])
# df.columns = [['c']*df.shape[1], df.columns]
# df
# df.addlevel?
# df.set_index?
# df.columns.levels
# pd.MultiIndex.from_product([df.columns, ['C']])

In [ ]:
A = S#[['predict']]
idx = list(A.index.names)
idx.remove('term_code')
idx
B = A.reset_index().pivot_table(
    index=idx,
    # values=['term_code','predict','current'],
    # values=['predict','current'],
    columns='term_code',
    # values='term_code',
)
C = B.swaplevel(0,1,1)
C[sorted(C.columns)]

In [ ]:
S.groupby('term_code')
S.pivot_table?

In [ ]:
self.Y.reset_index().dtypes

In [ ]:
self.X

In [ ]:
self.Y

In [ ]:
self.stack.keys()
# ['coll_desc']['mean'].keys()

In [ ]:
s = {3}
s.add(4)
s

In [ ]:
from LiveAMP import *
# A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/105/result/n/_allcrse/77d9/77d7/bb31.pkl')
A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/105/stack.pkl')
A.keys()

In [ ]:
self.get_X()
stack = {'Y': dict()}
for fn in sorted((self.root_path / 'result').rglob('*.pkl')):
    self.load(fn, force=True)
    self.load(str(fn).replace('result','Y'), force=True)
    stack['Y'][self.crse_code] = self.Y
    for k, v in self.result.items():
        for stat, df in v.items():
            stack.setdefault(k, dict()).setdefault(stat, dict())[self.crse_code] = df
Y = stack['Y']['_allcrse'].groupby(['pidm','pred_code']).agg(actual=('actual','mean'), proba_mean=('proba','mean'), proba_stdev=('proba','std'))
Z = self.X.join(Y).prep(bool=True)
# Z.to_excel(self.root_path / f'AMP_details_{self.cycle_date.date()}.xlsx')
Z.to_csv(self.root_path / f'AMP_details_{self.cycle_date.date()}.csv')

for k, v in stack.items():
    if k != 'Y':
        stack[k] = {stat: pd.concat(dct.values()) for stat, dct in v.items()}
    else:
        stack[k] = pd.concat(v.values())

with pd.ExcelWriter(self.root_path / f'AMP_{self.cycle_date.date()}.xlsx') as writer:
    for k, v in stack.items():
        if k != 'Y':
            v['mean'].droplevel(['trf_hash','imp_hash','clf_hash']).round(2).prep().to_excel(writer, sheet_name=k)

# stack['Z'] = self.X.join(Y).prep(bool=True)


In [ ]:
self.stack['Z']

In [ ]:
with pd.ExcelWriter(self.root_path / f'AMP_summary_{self.cycle_date.date()}.xlsx') as writer:
    

In [ ]:
# stack['crse_code']['summary']#['biol1407']
# stack['Y']
# details
self.stack['crse_code']['mean']#.keys()

In [ ]:
self.stack['crse_code']['biol1407'].keys()
# self.stack['Y'].keys()

In [ ]:
self.result['coll_desc']['mean']

In [ ]:
X = self.X.query(f"styp_code==@self.styp_code").rename(columns=lambda x:'__'+x)

In [ ]:
import dill
path = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/105/X.pkl'
with open(path, 'rb') as f:
    A = dill.load(f)


In [ ]:
for k,v in A.items():
    print(k, type(v))

In [ ]:
g = lambda y, nm: y.query(f"value=='{self.crse_code}'").droplevel('value').assign(**{nm:True})
Z = (
    self.X_proc
    .join(g(self.y[1],'registered'))
    .join(g(self.y[0],'actual'))
    .prep(bool=True, cat=True)
    .fillna({'registered':False,'actual':False})
)

train_code = 202108

stratify = ['actual','__act_equiv_missing','pidm']
grp = ['pred_code','sim']
qry = f"pred_code=={train_code}" if train_code != self.proj_code else f"pred_code!={train_code}"
X_model = (Z
    .query(qry)
    .sort_values(stratify, ascending=False).copy()
    .groupby(grp).filter(lambda x: x['actual'].sum() >= 5)
    .assign(msk = lambda x: x.groupby(grp).cumcount() % 5 > 0)
)
X_model['msk'] = X_model.groupby(grp).cumcount() % 5 > 0


In [ ]:
Z

In [ ]:
def intersection(*args, sort=False, **kwargs):
    L = [listify(x) for x in args]
    y = [x for x in L[0] if x in set(L[0]).intersection(*L)]
    return mysort(y, **kwargs) if sort else y

A = [2,1]
B = [1,2,3]
C = [2,4,1]
D = 1
intersection(A,B,C, sort=True)

In [ ]:
import dill, pathlib, pandas as pd
class T():
    def __init__(self):
        self.df = pd.DataFrame()
        self.df['A'] = [1,2,3]

t = T()
path = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/test_linux.pkl')
with open(path, 'wb') as file:
    dill.dump(t, file)

In [ ]:
self.get_X()
self.X

In [ ]:
from LiveAMP import *
T = Term(cycle_day=0, term_code=202308)
T.get_adm()
self = run_amp(119)

In [ ]:
self = AMP().load('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/119/X.pkl')
del self['path']
# del self['X']
# del self['Y']
# del self['y']
# del self['mlt']
# del self['aggregations']
del self['root_path']
self.dump('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/119/X.pkl')
self.__dict__.keys()

In [ ]:
# self.X.join(self.Y).groupby(['pred_code','pidm'])['proba'].agg(('mean','std'])
Y = self.stack['Y']['_allcrse'].groupby(['pidm','pred_code']).agg(actual=('actual','mean'), proba_mean=('proba','mean'), proba_stdev=('proba','std')).prep(bool=True)
# Z = self.Y.join(self.X)
# Z['proba'] *= 100
# var().sort_values(ascending=False)

In [ ]:
Z

In [ ]:
Z.groupby(['pred_code','pidm']).agg(actual=('actual','mean'), proba_mean=('proba','mean'), proba_stdev=('proba','std')).prep(bool=True)

In [ ]:
df = pd.DataFrame()
df['a'] = [2,1,pd.NA]
df.sort_values(['a','a'])
df.query('a>1')

In [ ]:
variable = 'gender'
self.mlt.query(f"variable=='{variable}'").droplevel(['variable']).rename_axis(index={'value':variable})

In [ ]:
from LiveAMP import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/119/rslt_camp_desc/n/_allcrse/77d9/77d7/fb12.pkl'
A = read(fn)

In [ ]:
from LiveAMP import *
# Flags().run()
self = run_amp(105)
1# self = AMP(cycle_day=105, styp_code='n', crse_code='_allcrse', param=cartesian(param_dcts)[0])
# self.get_result('gender')
# # self.get_Y()
# self.get_Y()

# # self.get_X_proc()
# self.get_X()

# variable = 'gender'
# self.get_result(variable)
# self.mlt.query(f"variable=='{variable}'").droplevel(['variable']).rename_axis(index={'value':variable})
# self.qry(self.mlt, variable).rename_axis(index={'value':variable})
# self.get_terms()
# self = run_amp(119, overwrite=[])

In [ ]:
import pathlib
root = pathlib.Path('/content')
# a = 'first'
a = ''
b = 'second'
c = root / a / b
str(c)


In [ ]:
x = 'HI'.lower()
x.

In [ ]:
self.root_path

In [ ]:
for k,v in self.__dict__.items():
    print(k, type(v), isinstance(v, pathlib.PosixPath))


In [ ]:
self.X.missing().disp(100)

In [ ]:
self.y[0].vc('pred_code')
self.X.vc('pred_code')

In [ ]:
self.X.vc('pred_code')

In [ ]:
self.Y.vc('pred_code')

In [ ]:
variable = 'crse_code'
variable = 'gender'
grp = uniquify([variable,'levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim'])
S = (
    self.X
    .join(self.Y, how='inner')
    # .reset_index()
    .groupby(grp).apply(lambda y: pd.Series({
            # 'pred_code_final': y['actual'].sum(),
            'pred_code_predict': y['proba'].sum(),
            'test_score': log_loss(y['actual'], y['proba'], labels=[False,True]) * 100,
        }), include_groups=False)
    .join(self.train_score)
    .join(self.mlt.query(f"variable=='{variable}'").droplevel('variable').rename_axis(index={'value':variable}))
)
alpha = 1
S['overall_score'] = (S['train_score'] + alpha * S['test_score']) / (1 + alpha)
# qry = lambda q: S.query(f"pred_code=={q}").droplevel('pred_code')
# S = (S
#     .join(qry(self.proj_code-100)['pred_code_current'].rename(prior_current))
#     .join(qry(self.proj_code    )['pred_code_current'].rename(proj_current))
#     .join(qry(self.proj_code-100)['pred_code_final'  ].rename(prior_final))
#     .join(qry(self.proj_code    )['pred_code_predict'].rename(proj_predict))
#     .query(f"pred_code!={self.proj_code}")
#     .astype('Float64').fillna(0)
# )

S.vc('pred_code')

In [ ]:
self.mlt

In [ ]:
self.mlt.query(f"variable=='{variable}'").droplevel('variable').rename_axis(index={'value':variable})

In [ ]:
self.mlt

In [ ]:
g = lambda y, nm: y.query(f"value=='{self.crse_code}'").droplevel('value').assign(**{nm:True})
g(self.y[1], 'hi')

In [ ]:
self.y_final

In [ ]:
a = self.y_current.rsindex(['pidm','pred_code','value']).assign(current=True)
a
# crse_code = '_allcrse'
# a.query(f"value=='{crse_code}'").droplevel('value').assign(current=True)
# self.X

In [ ]:
self.mlt

In [ ]:
self.mlt

In [ ]:
variable = 'crse_code'
variable = 'gender'
grp = uniquify([variable,'levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim'])
S = (
    self.X
    .join(self.Y, how='inner')
    # .reset_index()
    .groupby(grp).apply(lambda y: pd.Series({
            # 'pred_code_final': y['actual'].sum(),
            'pred_code_predict': y['proba'].sum(),
            'test_score': log_loss(y['actual'], y['proba'], labels=[False,True]) * 100,
        }), include_groups=False)
    .join(self.train_score)
    # .join(self.qry(self.mlt, variable).rename_axis(index={'value':variable}).rename(columns=lambda x: "pred_code_"+x))
    # .join(self.mlt.rename(columns=lambda x: "pred_code_"+x))
)
S

In [ ]:
variable = 'gender'
self.qry(self.mlt, variable)#.rename_axis(index={'value':variable}).rename(columns=lambda x: "pred_code_"+x)

self.mlt.query(f"variable=='{variable}'").droplevel('variable').rename_axis(index={'value':variable})

In [ ]:
self.mlt

In [ ]:

self.X.join(self.Y)

In [ ]:
self.Y

In [ ]:
self.X.disp()

In [ ]:
self.get_X()
self.X.disp()

In [ ]:
self.y_current

In [ ]:
self.y_final.dtype

In [ ]:
qry = lambda y, value: y.query(f"variable=='{value}'").droplevel('variable')
g = lambda y, nm: qry(y, self.crse_code).rename(nm)
Z = (
    self.X_proc
    .join(g(self.y_current, 'registered'))
    .join(g(self.y_final, 'actual'))
    .prep(bool=True, cat=True)
    .fillna({'registered':False, 'actual':False})
    # .sort_index()
)
# g = lambda y: self.qry(y, self.crse_code)


# g = lambda y: y.query(f"variable=='{self.crse_code}'").droplevel('variable')

# if 'registration' in dct and dct.pop('registration'):
#     Z = Z.join(g(self.y['current']))


# if 'registration' in dct and dct.pop('registration'):
#     Z = Z.join(self.credit).join(g(self.y['current']))
# Z = (Z
#     .join(g(self.y['admit'].rename('actual')))
#     .fillna({'credit':0, 'current':False, 'actual':False})
#     # .fillna({self.crse_code:False, 'current':False, 'credit':0})
#     .prep(bool=True, cat=True)
#     .sort_index()
# )
Z.vc('registered')
Z

In [ ]:
self.X

In [ ]:
import dill, dataclasses, pandas as pd

# @dataclasses.dataclass
class MyClass():
    def __init__(self):
        self.df = pd.DataFrame()
        self.df['A'] = [1,2,3]
        self.df['B'] = ['kai','q','mumford']

obj = MyClass()
path = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/test2.pkl'
with open(path, 'wb') as f:
    dill.dump(obj, f)


In [ ]:
obj.df

In [ ]:
import dill
path = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/test.pkl'
with open(path, 'rb') as f:
    obj = dill.load(f)
obj.df

In [ ]:
self.mlt

In [ ]:
self.X[1][[]].join(self.Y[0].droplevel(col)).query(qry).groupby(['pred_code','levl_code','styp_code','variable','value']).size()

In [ ]:
self.X[1].melt(ignore_index=False)

In [ ]:
col = ['levl_code','styp_code']
qry = f"levl_code == 'ug' & styp_code in ('n','r','t') & pred_code!={self.proj_code}"
variables = ['gender','lgcy']
aggy = lambda y: y.query(qry).groupby(['pred_code','levl_code','styp_code','variable','value']).size()
aggx = lambda x: aggy(x[variables].melt(ignore_index=False))

def get_df(dct):
    Y = pd.DataFrame(dct).fillna(0).prep()
    Y['mlt'] = Y['final'] / Y['admitted']
    Y[np.isinf(Y)] = pd.NA
    return Y

AY = get_df({
    'current': aggy(self.Y[1]),
    'admitted': aggy(self.X[1][[]].join(self.Y[0].droplevel(col))),
    'final': aggy(self.Y[0]),
})

AX = get_df({
    'current': aggx(self.X[1].query("current.notnull()")),
    'admitted': aggx(self.X[1].query("final.notnull()")),
    'final': aggx(self.X[0].query("final.notnull()")),
})

pd.concat([AX,AY])

In [ ]:
self.X[1].disp()

In [ ]:
final

In [ ]:
self.X[0]

In [ ]:
self.X[0].vc('value')

In [ ]:
self.Y[1]

In [ ]:
self.Y[0]

In [ ]:
attr = ['gender','lgcy']
grp = ['pred_code','levl_code','styp_code','variable','value']
# X = self.X[1][attr].melt(ignore_index=False)
aggy = lambda y: y.groupby(grp).size()
aggx = lambda x: aggy(x[attr].droplevel('value').melt(ignore_index=False))
current = aggx(self.X[1].query('value.notnull()'))
final = aggx(self.X[0])
# current = self.X[1].query("credit_hr>0")[attr].melt(ignore_index=False).groupby(grp).size()
X = self.X[0].droplevel('value').join(self.X[1][[]])
admitted = aggx(X.query("value.notnull()"))



current = aggy(self.Y[1])
final = aggy(self.Y[0])
Y = self.Y[0].droplevel('value').join(self.Y[1][[]])
admitted = aggy(Y.query("value.notnull()"))
final = aggy(Y)

# final = X.groupby(grp).size()

# current = self.Y[1].query("enrolled_ind").melt(ignore_index=False).groupby(grp).size()

# self.X[1].vc('enrolled_ind')

# current#.disp(4)
# Y
# admitted
final.query("variable=='_allcrse'")
# self.X[1]
# X

In [ ]:
self.Y[1].vc('value')

In [ ]:
aggy(self.Y[0])

In [ ]:
self.Y[0]

In [ ]:
# self.X[1].melt(ignore_index=False).droplevel(['id','idx'])
grp = ['pred_code','levl_code','styp_code','variable','value']
col = ['gender','lgcy']
aggy = lambda y: y.groupby(grp).size()
aggx = lambda x: aggy(x[col].melt(ignore_index=False))
aggx(self.X[0])
aggx(self.X[0].join(self.X[1][[]].rsindex(['pidm','pred_code']), how='inner'))
# aggy(self.Y[1])
# self.X[0]
# self.X[0]['credit_hr'].min()
# self.X[1]
# self.X[0][[]].rsindex(['pidm','pred_code'])

In [ ]:
a = self.y0.query(f"styp_code=='n' & variable=='_allcrse' & credit_hr>0").groupby(['pred_code','styp_code']).size()
# b = self.Y[0].query(f"variable=='_allcrse' & styp_code=='n'").groupby(['pred_code','styp_code']).sum()
print(a)
self.X0.query(f"styp_code=='n' & enrolled_ind").groupby(['pred_code','styp_code']).size()

In [ ]:
y.vc('styp_code')

In [ ]:
self.get_terms()
self.terms[0][202308].cycle_date

In [ ]:
# qry = f"select * from opeir.registration_fall2023 A where A.current_date={dt('2023-09-13')} and pidm=1108765"
qry = f"select * from opeir.registration_fall2021 A where pidm=1026466"
# 1111252"
db.execute(qry).disp(1000)

In [ ]:
y = self.y0.to_frame().query(f"variable=='_allcrse' & credit_hr>0 & pred_code<202408").reset_index(['levl_code','styp_code','variable'], drop=True)
# y = self.y0.to_frame().query(f"variable=='_allcrse'").reset_index('variable', drop=True).reset_index(['levl_code','styp_code'])
X = self.X0.query(f"styp_code=='n' & enrolled_ind")[['enrolled_ind']]
Z = X.join(y, how="left")
Z[Z.isnull().any(axis=1)]#.vc('pred_code')
# y.join(X).vc(['pred_code','styp_code'])

In [ ]:
y = self.y0.to_frame().query(f"variable=='_allcrse' & credit_hr>0 & levl_code=='ug' & styp_code=='n'").reset_index(['levl_code','styp_code','variable'], drop=True)
# y = self.y0.to_frame().query(f"variable=='_allcrse'").reset_index('variable', drop=True).reset_index(['levl_code','styp_code'])
X = self.X0#.query(f"styp_code=='n' & enrolled_ind")[[]]
# Z = X.join(y, how="left")
# Z[Z.isnull().any(axis=1)]
y.join(X).vc(['pred_code','styp_code'])

In [ ]:
self.get_X()
self.mlt

In [ ]:
self.Y

In [ ]:
self.Y

In [ ]:
qry = f"""
select
    A.*,
    (select B.stvlgcy_desc from stvlgcy B where B.stvlgcy_code = A.lgcy_code) as lgcy_desc
from (
    select
        A.*,
        (select B.spbpers_lgcy_code from spbpers B where B.spbpers_pidm = A.pidm) as lgcy_code
    from
        opeir.admissions_fall2023 A
    where enrolled_ind='Y' and A.current_date = {dt('2023-09-13')}
) A
--fetch first 1 rows only
"""
X = db.execute(qry)

In [ ]:
X.vc(['lgcy_code','lgcy_desc'])
# X['lgcy_code'].isin([pd.NA,'o']).sum()

In [ ]:
self.mlt

In [ ]:
self.y['final']
# self.credit

In [ ]:
self.X.vc(['pred_code','lgcy'])

In [ ]:
self.credit
g = lambda y: y.query(f"variable=='{self.crse_code}'").droplevel('variable')
y = self.y['current']
y.squeeze().rename_axis(index={'variable':'crse_code'}).rename('d')
# g(self.y['current'])

In [ ]:
# for k,v in self.y.items():
#     print(k)
#     v.disp(4)
self.mlt.query("styp_code=='n'").head(10)
# self.__dict__.keys()
# self.credit

In [ ]:
self.X.apdc_code

In [ ]:
attr = ['gender','lgcy']
Z = self.X_final[attr].melt(ignore_index=False).astype('string')
A1 = Z.groupby(['styp_code','pred_code','variable','value']).size()

In [ ]:
ren = {'term_code':'pred_code', 'term_desc':'pred_desc', 'index':'idx', 'crse_code':'variable'}
Y = [pd.concat([
        T.reg
        .rename(columns=ren)
        .set_index(['pidm','levl_code','styp_code','pred_code','variable'])
        ['credit_hr']
        .fillna(0)
        .query('credit_hr>0')
        .rename('value')
        .to_frame()
        > 0
        
    for pred_code, T in dct.items()]) for dct in self.terms]
Y[1].rename(columns={'a':'b'})
# A2 = Y[1].groupby(['styp_code','pred_code','variable','value']).size()

In [ ]:
pd.concat([A1,A2])

In [ ]:
dct = self.terms[1]
ren = {'term_code':'pred_code', 'term_desc':'pred_desc', 'index':'idx', 'crse_code':'variable'}
R = pd.concat([T.raw.rename(columns=ren) for pred_code, T in dct.items()]).copy().dropna(axis=1, how='all').reset_index(drop=True).prep()
repl = {'ae':0, 'n1':1, 'n2':2, 'n3':3, 'n4':4, 'r1':1, 'r2':2, 'r3':3, 'r4':4}
R['hs_qrtl'] = pd.cut(R['hs_pctl'], bins=[-1,25,50,75,90,101], labels=[4,3,2,1,0], right=False).combine_first(R['apdc_code'].map(repl))
R['remote'] = R['camp_code'] != 's'
R['resd'] = R['resd_code'] == 'r'
R['oriented'] = R['oriented'] != 'n'
R['lgcy'] = ~R['lgcy_code'].isin(['n','o'])
R['majr_code'] = R['majr_code'].replace({'0000':pd.NA, 'und':pd.NA, 'eled':'eted', 'agri':'unda'})
R['coll_code'] = R['coll_code'].replace({'ae':'an', 'eh':'ed', 'hs':'hl', 'st':'sm', '00':pd.NA})
R['coll_desc'] = R['coll_code'].map({
    'an': 'ag & natural_resources',
    'ba': 'business',
    'ed': 'education',
    'en': 'engineering',
    'hl': 'health sciences',
    'la': 'liberal & fine arts',
    'sm': 'science & mathematics',
    pd.NA: 'no college designated',
})
majr = ['majr_desc','dept_code','dept_desc','coll_code','coll_desc']
S = R.sort_values('cycle_date').drop_duplicates(subset='majr_code', keep='last')[['majr_code',*majr]]
X = R.drop(columns=majr).merge(S, on='majr_code', how='left').prep(bool=True)
checks = [
    'cycle_day >= 0',
    'apdc_day >= cycle_day',
    'appl_day >= apdc_day',
    'birth_day >= appl_day',
    'birth_day >= 5000',
    'distance >= 0',
    'hs_pctl >=0',
    'hs_pctl <= 100',
    'hs_qrtl >= 0',
    'hs_qrtl <= 4',
    'act_equiv >= 1',
    'act_equiv <= 36',
    'gap_score >= 0',
    'gap_score <= 100',
]
for check in checks:
    mask = X.eval(check)
    assert mask.all(), [check,X[~mask].disp(5)]

fill = {
    'birth_day': ['median',['pred_code','styp_code']],
    'remote': False,
    'international': False,
    **{f'race_{r}': False for r in ['american_indian','asian','black','pacific','white','hispanic']},
    'lgcy': False,
    'resd': False,
    'waiver': False,
    'fafsa_app': False,
    'schlship_app': False,
    'finaid_accepted': False,
    'ssb': False,
    'math': False,
    'reading': False,
    'writing': False,
    'gap_score': 0,
    'oriented': False,
    # 'coll_code': 'na',
    # 'coll_desc': 'na',
}
for k, v in fill.items():
    X[k] = X.impute(k, *listify(v))
M = X.isnull().rename(columns=lambda x:x+'_missing')
attr = ['idx','id','pidm','levl_code','styp_code','pred_code']
X = X.join(M).sample(frac=1, random_state=self.param['imp'][2]['random_state']).reset_index().rename(columns=ren).prep(bool=True).set_index(attr).sort_index()
X.disp(4)

In [ ]:
X

In [ ]:
y = pd.concat([
        T.reg
        .rename(columns=ren)
        .set_index(['pidm','pred_code','variable'])
        # .set_index(['pidm','pred_code','crse_code'])
        [['credit_hr','levl_code','styp_code']]
        # .query('credit_hr>0')
        # .rename('value')
    for pred_code, T in dct.items()])#.reset_index()#.unstack().fillna(0)#.prep()
y

In [ ]:
y

In [ ]:
cols = ['levl_code_x','levl_code_y','styp_code_x','styp_code_y']
Z = y.reset_index().merge(X[[]].reset_index(), on=['pidm','pred_code'], how='outer').fillna({k:'' for k in cols})
# Z = Z[['levl_code_x','levl_code_y','styp_code_x','styp_code_y']].fillna('')
msk = (Z['levl_code_x'] != Z['levl_code_y']) | (Z['styp_code_x'] != Z['styp_code_y'])
Z[msk].vc(cols).sort_index().reset_index()
# Z

In [ ]:
Z = X[[]].join(y, how='right').reset_index()

In [ ]:
y.reset_index()

In [ ]:
y.query("variable=='_allcrse'").drop(columns='variable')

In [ ]:
r = y.query("variable=='_allcrse' & value>0").droplevel('variable').rename('registered')>0
r

In [ ]:
y.to_frame().join(r)

In [ ]:
attr = ['gender','lgcy']
Z = X[attr].melt(ignore_index=False).astype('string').join(r)
Z

In [ ]:
y.join(y.query("variable=='_allcrse'").droplevel('variable').rename('registered')>0)

In [ ]:
Z.groupby(['styp_code','pred_code','variable','value'])['registered'].sum()

In [ ]:
Z = X.join(y).melt(ignore_index=False).astype('string')

In [ ]:
Z.dtypes

In [ ]:
Z.prep().dtypes

In [ ]:
pd.concat([X.melt(ignore_index=False).reset_index(), y])

In [ ]:
self.X

In [ ]:
self.get_X()
self.mlt.query("crse_code=='_allcrse'")

In [ ]:
A = self.summary.groupby('crse_code')['pred_code_actual'].agg(['min','max'])
A.query('min>1 & min<5').sort_values('max', ascending=False)

In [ ]:
import numpy as np
x = np.arange(100)
100 * (1 - (x%5==0).cumsum() / (x+1))

In [ ]:
self.train_score
self.get_X()
nm='crse_code'
grp = uniquify([nm,'crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim'])
Z = self.X.join(self.Y, how='inner').reset_index()
S = Z.groupby(grp).apply(lambda y: pd.Series({
    'pred_code_actual': y['actual'].sum(),
    'pred_code_predicted': y['proba'].sum(),
    'test_score': log_loss(y['actual'], y['proba'], labels=[False,True]) * 100,
}), include_groups=False).join(self.train_score)


In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt3/119/rslt_crse_code/n/_allcrse/fa15/77d7/d6c4.pkl')
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt5/119/rslt_crse_code/n/_allcrse/fa15/77d7/bf3e.pkl')
print(w['param']['clf'][2]['time_budget'])
w['rslt_crse_code'][' 50%']

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt4/119/rslt_crse_code/n/_allcrse/fa15/77d7/d6c4.pkl')
w['rslt_crse_code'][' 50%']

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt5/119/rslt_crse_code/n/_allcrse/fa15/77d7/b66c.pkl')
w['rslt_crse_code'][' 50%']

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/119/rslt_crse_code/n/_allcrse/fa15/77d7/114b.pkl')
w['rslt_crse_code'][' 50%']

In [ ]:
y1 = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/119/Y/n/_allcrse/fa15/77d7/b66c.pkl')['Y']
y2 = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/119/Y/n/_allcrse/fa15/77d7/b66c.pkl')['Y']
y2b = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/119/Y/n/_allcrse/fa15/77d7/114b.pkl')['Y']
y3 = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt3/119/Y/n/_allcrse/fa15/77d7/b66c.pkl')['Y']
nm = 'crse_code'
def g(y):
    grp = uniquify([nm,'crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim'])
    z = y.groupby(grp)[['actual','proba']].sum()
    grp.remove('sim')
    return z.groupby(grp).mean()

In [ ]:
g(y1)

In [ ]:
g(y2)

In [ ]:
g(y2b)

In [ ]:
g(y3)

In [ ]:
g(y3) - g(y2)